In [83]:
import pymysql                                                                                                                                                
import pandas as pd
import numpy as np
import itertools
import scipy.sparse.linalg

In [84]:
def sql_result(query):
    database = pymysql.connect(host='db-read.truebil.com', user='truebil-read',
                               passwd='newreadpassword', db='truebil')
    cursor = database.cursor()
    cursor.execute(query)
    database.commit()
    query_result_list = [row for row in cursor.fetchall()]
    try:
        field_names = [i[0] for i in cursor.description]
        query_result_dataframe = pd.DataFrame(data=query_result_list, columns=field_names)
    except:
        query_result_dataframe = None
    return query_result_dataframe

In [85]:
# markov_query = """
# (SELECT b.id as buyer_id, us.created_at, 1 AS visit_source FROM user_seens us
# JOIN users u ON us.user_id = u.id JOIN buyers b ON u.mobile = b.mobile WHERE us.created_at >= (NOW() - INTERVAL 2 MONTH)
# AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
# UNION
# (SELECT bl.buyer_id, bl.created_at, 2 AS visit_source FROM buyer_listings bl JOIN buyers b ON b.id = bl.buyer_id
# WHERE bl.source_id NOT IN (2, 8, 42) AND bl.created_at >= (NOW() - INTERVAL 2 MONTH) AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
# UNION
# (SELECT bl.buyer_id, bl.created_at, 3 AS visit_source FROM buyer_listing_offers bl JOIN buyers b ON b.id = bl.buyer_id
# WHERE bl.created_at >= (NOW() - INTERVAL 2 MONTH) AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
# UNION
# (SELECT b.id, bvl.created_at, 4 AS visit_source FROM buyer_visit_listings bvl JOIN buyer_visits bv
# ON bv.id = bvl.buyer_visit_id JOIN buyers b ON bv.buyer_id = b.id WHERE bvl.source_id in (4, 6, 17, 18, 19)
# AND bvl.created_at >= (NOW() - INTERVAL 2 MONTH) AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
# UNION
# (SELECT b.id, bvltdl.created_at, 5 AS visit_source FROM buyer_visit_listing_test_driven_logs bvltdl
# JOIN buyer_visit_listings bvl ON bvltdl.buyer_visit_listing_id = bvl.id JOIN buyer_visits bv ON
# bv.id = bvl.buyer_visit_id JOIN buyers b ON bv.buyer_id = b.id WHERE bvltdl.buyer_visit_listing_test_driven_status_id =
# 9 AND bvltdl.created_at >= (NOW() - INTERVAL 2 MONTH) AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
# UNION
# (SELECT b.id, bvltdl.created_at, 6 AS visit_source FROM buyer_visit_listing_test_driven_logs bvltdl
# JOIN buyer_visit_listings bvl ON bvltdl.buyer_visit_listing_id = bvl.id JOIN buyer_visits bv
# ON bv.id = bvl.buyer_visit_id JOIN buyers b ON bv.buyer_id = b.id
# WHERE bvltdl.buyer_visit_listing_test_driven_status_id = 1 AND bvltdl.created_at >= (NOW() - INTERVAL 2 MONTH)
# AND b.created_at >= (NOW() - INTERVAL 1 YEAR))"""

# buyer_data = sql_result(markov_query)

In [86]:
markov_query = """
(SELECT b.id as buyer_id, us.created_at, 'seen' AS visit_source FROM user_seens us
JOIN users u ON us.user_id = u.id JOIN buyers b ON u.mobile = b.mobile WHERE us.created_at >= (NOW() - INTERVAL 2 MONTH)
AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
UNION
(SELECT bl.buyer_id, bl.created_at, 'crf' AS visit_source FROM buyer_listings bl JOIN buyers b ON b.id = bl.buyer_id
WHERE bl.source_id NOT IN (2, 8, 42) AND bl.created_at >= (NOW() - INTERVAL 2 MONTH) AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
UNION
(SELECT bl.buyer_id, bl.created_at, 'mo' AS visit_source FROM buyer_listing_offers bl JOIN buyers b ON b.id = bl.buyer_id
WHERE bl.created_at >= (NOW() - INTERVAL 2 MONTH) AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
UNION
(SELECT b.id, bvl.created_at, 'vb' AS visit_source FROM buyer_visit_listings bvl JOIN buyer_visits bv
ON bv.id = bvl.buyer_visit_id JOIN buyers b ON bv.buyer_id = b.id WHERE bvl.source_id in (4, 6, 17, 18, 19)
AND bvl.created_at >= (NOW() - INTERVAL 2 MONTH) AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
UNION
(SELECT b.id, bvltdl.created_at, 'vd' AS visit_source FROM buyer_visit_listing_test_driven_logs bvltdl
JOIN buyer_visit_listings bvl ON bvltdl.buyer_visit_listing_id = bvl.id JOIN buyer_visits bv ON
bv.id = bvl.buyer_visit_id JOIN buyers b ON bv.buyer_id = b.id WHERE bvltdl.buyer_visit_listing_test_driven_status_id =
9 AND bvltdl.created_at >= (NOW() - INTERVAL 2 MONTH) AND b.created_at >= (NOW() - INTERVAL 1 YEAR))
UNION
(SELECT b.id, bvltdl.created_at, 'sold' AS visit_source FROM buyer_visit_listing_test_driven_logs bvltdl
JOIN buyer_visit_listings bvl ON bvltdl.buyer_visit_listing_id = bvl.id JOIN buyer_visits bv
ON bv.id = bvl.buyer_visit_id JOIN buyers b ON bv.buyer_id = b.id
WHERE bvltdl.buyer_visit_listing_test_driven_status_id = 1 AND bvltdl.created_at >= (NOW() - INTERVAL 2 MONTH)
AND b.created_at >= (NOW() - INTERVAL 1 YEAR))"""

buyer_data = sql_result(markov_query)

In [87]:
invalid_buyers_query = """SELECT b.id FROM buyers b JOIN invalid_numbers i ON b.mobile = i.mobile"""
invalid_buyers = sql_result(invalid_buyers_query)
invalid_buyers_list = invalid_buyers['id'].tolist()

In [172]:
buyer_data

,buyer_id,created_at,visit_source,visit_order,visit
509896,852417,2020-03-13 08:22:17,crf,1,2
509895,852416,2020-03-13 08:20:39,mo,1,3
509893,852415,2020-03-13 08:19:39,crf,1,2
509894,852415,2020-03-13 08:19:56,seen,2,1
509890,852414,2020-03-13 08:18:45,crf,1,2
509891,852414,2020-03-13 08:19:37,vb,2,4
509892,852414,2020-03-13 08:20:13,seen,3,1
509888,852413,2020-03-13 08:18:30,crf,1,2
509889,852413,2020-03-13 08:22:25,seen,2,1
509886,852411,2020-03-13 08:09:50,crf,1,2


In [90]:
buyer_data = buyer_data[~buyer_data['buyer_id'].isin(invalid_buyers_list)]
buyer_data = buyer_data.sort_values(['buyer_id', 'created_at'],ascending=[False, True])
buyer_data['visit_order'] = buyer_data.groupby('buyer_id').cumcount() + 1

In [91]:
def get_interaction_id(row):
    INTERACTION_DICT = {'seen': 1, 'crf': 2, 'mo': 3, 'vb': 4, 'vd': 5, 'sold': 6}
    interaction_id = INTERACTION_DICT[row['visit_source']]
    return interaction_id

In [92]:
buyer_data['visit'] = buyer_data.apply(lambda x: get_interaction_id(x), axis=1)

In [93]:
k = buyer_data

In [94]:
visit_order_data = buyer_data.groupby('buyer_id')['visit'].aggregate(lambda x: [1] + x.tolist()).reset_index()
visit_order_data.rename(columns = {'visit':'visit_transitions'}, inplace = True) 
# visit_order_data = visit_order_data[visit_order_data.visit_transitions..len() > 2].reset_index(drop=True)

visit_order_data = visit_order_data[visit_order_data['visit_transitions'].apply(lambda x: len(x) > 2)].reset_index(drop=True)

In [95]:
visit_order_data

,buyer_id,visit_transitions
0,418415,"[1, 1, 2, 4, 1, 1, 2, 4, 1, 2, 4, 1, 4, 2, 2, ..."
1,447115,"[1, 2, 1, 1, 1, 1, 1, 1, 1]"
2,473455,"[1, 3, 3]"
3,473489,"[1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, ..."
4,473493,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,473547,"[1, 1, 1]"
6,473633,"[1, 1, 1]"
7,473689,"[1, 1, 1]"
8,473792,"[1, 3, 1, 5]"
9,473924,"[1, 1, 1, 1]"


In [96]:
def get_trans_prob(visit_transitions):
    transition_rank_list = [transition - 1 for transition in visit_transitions]
    trans_prob_list = [[0] * 6 for _ in range(6)]

    for (i, j) in zip(transition_rank_list, transition_rank_list[1:]):
        trans_prob_list[i][j] += 1

    for data in trans_prob_list:
        list_sum = sum(data)
        if list_sum > 0:
            data[:] = [unit / sum(data) for unit in data]

    return trans_prob_list

In [105]:
get_trans_prob([1, 2, 1, 1, 1, 1, 1, 1, 1])

[[0.8571428571428571, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

In [106]:
visit_order_data['trans_prob'] = visit_order_data['visit_transitions'].apply(lambda x: get_trans_prob(x))

In [107]:
trans_prob_data = visit_order_data['trans_prob'].tolist()
merged_prob_data = list(itertools.chain.from_iterable(trans_prob_data))

In [108]:
trans_prob_data

[[[0.7631578947368421,
   0.21052631578947367,
   0.0,
   0.02631578947368421,
   0.0,
   0.0],
  [0.07692307692307693,
   0.15384615384615385,
   0.0,
   0.7692307692307693,
   0.0,
   0.0],
  [0, 0, 0, 0, 0, 0],
  [0.6363636363636364, 0.2727272727272727, 0.0, 0.09090909090909091, 0.0, 0.0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 [[0.8571428571428571, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0],
  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
  [0, 0, 0, 0, 0, 0],
  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 [[0.8, 0.2, 0.0, 0.0, 0.0, 0.0],
  [0.6, 0.4, 0.0, 0.0, 0.0, 0.0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0]],
 [[1.0, 0.0, 0

In [109]:
len(merged_prob_data)

294726

In [110]:
# def get_apt_prob_data(merged_prob_data, index):
#     apt_prob_data = []
#     limit = len(merged_prob_data) - index
#     flag = 6 - index
#     transistion_count = 0
#     while flag <= limit:
#         if sum(merged_prob_data[flag]) != 0:
#             transistion_count += 1
#             apt_prob_data.append(merged_prob_data[flag])
#         flag += 6

#     return apt_prob_data, transistion_count


# def get_average_prob_data(merged_prob_data):
#     index = 6
#     average_prob_data = []
#     avg_transistion_count = []
#     while index > 0:
#         apt_prob_data, transistion_count = get_apt_prob_data(merged_prob_data, index)
#         avg_transistion_count.append(transistion_count)
#         apt_prob_data = np.array(apt_prob_data)
#         apt_prob_data = np.average(apt_prob_data, axis=0).tolist()
#         average_prob_data.append(apt_prob_data)
#         index -= 1

#     return average_prob_data, avg_transistion_count

In [127]:
def get_apt_prob_data(merged_prob_data, visit_transitions_data, index):
    test_dict = {6: 1, 5: 2, 4: 3, 3: 4, 2: 5, 1: 6}
    apt_prob_data = []
    limit = len(merged_prob_data) - index
    flag = 6 - index
    transistion_count = []
    while flag <= limit:
        if sum(merged_prob_data[flag]) != 0:
            t = flag / 6
            user_transitions = visit_transitions_data[int(t)]
            transistion_count.append(user_transitions.count(6-index+1))
            apt_prob_data.append(merged_prob_data[flag])
        flag += 6
    
    return apt_prob_data, transistion_count


def get_average_prob_data(merged_prob_data, visit_transitions_data):
    index = 6
    average_prob_data = []
    avg_transistion_count = []
    while index > 0:
        apt_prob_data, transistion_count = get_apt_prob_data(merged_prob_data, visit_transitions_data, index)
        avg_transistion_count.append(transistion_count)
        apt_prob_data = np.array(apt_prob_data)
        apt_prob_data = np.average(apt_prob_data, axis=0).tolist()
        average_prob_data.append(apt_prob_data)
        index -= 1

    return average_prob_data, avg_transistion_count

In [128]:
average_prob_data, avg_transistion_count = get_average_prob_data(merged_prob_data, visit_transitions_data)

In [129]:
avg_transistion_count

[[38,
  8,
  1,
  16,
  15,
  3,
  3,
  3,
  2,
  4,
  11,
  3,
  16,
  4,
  4,
  5,
  8,
  4,
  4,
  3,
  14,
  7,
  11,
  3,
  9,
  5,
  12,
  37,
  47,
  3,
  5,
  7,
  1,
  29,
  5,
  1,
  5,
  4,
  3,
  2,
  39,
  41,
  96,
  10,
  3,
  4,
  6,
  4,
  3,
  3,
  6,
  3,
  3,
  5,
  4,
  8,
  7,
  3,
  5,
  9,
  10,
  4,
  10,
  29,
  6,
  16,
  24,
  3,
  1,
  3,
  6,
  2,
  11,
  42,
  1,
  2,
  3,
  38,
  3,
  9,
  77,
  4,
  29,
  3,
  6,
  7,
  13,
  6,
  19,
  1,
  17,
  5,
  4,
  10,
  2,
  15,
  25,
  1,
  8,
  2,
  23,
  19,
  7,
  4,
  2,
  6,
  27,
  41,
  54,
  5,
  5,
  1,
  23,
  3,
  3,
  9,
  13,
  10,
  2,
  40,
  8,
  1,
  7,
  11,
  263,
  1,
  20,
  5,
  16,
  13,
  5,
  1,
  2,
  2,
  11,
  16,
  29,
  4,
  20,
  4,
  15,
  8,
  4,
  4,
  16,
  1,
  46,
  23,
  6,
  24,
  5,
  18,
  4,
  49,
  7,
  8,
  13,
  3,
  3,
  7,
  5,
  10,
  55,
  14,
  4,
  15,
  5,
  27,
  3,
  4,
  3,
  6,
  9,
  8,
  1,
  2,
  12,
  5,
  90,
  4,
  23,
  3,
  2,
  3,
  25,
  33,
  

In [130]:
avg_transistion_count

[[38,
  8,
  1,
  16,
  15,
  3,
  3,
  3,
  2,
  4,
  11,
  3,
  16,
  4,
  4,
  5,
  8,
  4,
  4,
  3,
  14,
  7,
  11,
  3,
  9,
  5,
  12,
  37,
  47,
  3,
  5,
  7,
  1,
  29,
  5,
  1,
  5,
  4,
  3,
  2,
  39,
  41,
  96,
  10,
  3,
  4,
  6,
  4,
  3,
  3,
  6,
  3,
  3,
  5,
  4,
  8,
  7,
  3,
  5,
  9,
  10,
  4,
  10,
  29,
  6,
  16,
  24,
  3,
  1,
  3,
  6,
  2,
  11,
  42,
  1,
  2,
  3,
  38,
  3,
  9,
  77,
  4,
  29,
  3,
  6,
  7,
  13,
  6,
  19,
  1,
  17,
  5,
  4,
  10,
  2,
  15,
  25,
  1,
  8,
  2,
  23,
  19,
  7,
  4,
  2,
  6,
  27,
  41,
  54,
  5,
  5,
  1,
  23,
  3,
  3,
  9,
  13,
  10,
  2,
  40,
  8,
  1,
  7,
  11,
  263,
  1,
  20,
  5,
  16,
  13,
  5,
  1,
  2,
  2,
  11,
  16,
  29,
  4,
  20,
  4,
  15,
  8,
  4,
  4,
  16,
  1,
  46,
  23,
  6,
  24,
  5,
  18,
  4,
  49,
  7,
  8,
  13,
  3,
  3,
  7,
  5,
  10,
  55,
  14,
  4,
  15,
  5,
  27,
  3,
  4,
  3,
  6,
  9,
  8,
  1,
  2,
  12,
  5,
  90,
  4,
  23,
  3,
  2,
  3,
  25,
  33,
  

In [131]:
average_prob_data

[[0.5279314471649116,
  0.35312105150144185,
  0.10458844461162081,
  0.0027177272684323415,
  0.01141612431042796,
  0.00022520514315489288],
 [0.6507207348073527,
  0.06058925795815044,
  0.045531498253299066,
  0.2316192524026409,
  0.011451533450871715,
  8.772312768340108e-05],
 [0.7268503623441341,
  0.13169685735454117,
  0.10187637665619122,
  0.028272218006492,
  0.011251454147395656,
  5.273149124657245e-05],
 [0.794586472539281,
  0.1262060852211035,
  0.05553300914826999,
  0.010744147668022412,
  0.012591144000981463,
  0.00033914142234214975],
 [0.4847052845528458,
  0.08046602787456447,
  0.06784988385598141,
  0.003048780487804878,
  0.12352227476356395,
  0.2404077484652396],
 [0.815840621963071,
  0.08892128279883382,
  0.028182701652089404,
  0.0029154518950437317,
  0.04956268221574344,
  0.014577259475218658]]

In [132]:
required_data = []
for data in avg_transistion_count:
    l = [sum(data), np.mean(data), np.var(data), np.std(data)]
    required_data.append(l)

In [133]:
required_data

[[425491, 8.662099712953726, 184.1205091719551, 13.569101266183958],
 [60834, 1.7770053163521644, 2.923983717504014, 1.7099659989321467],
 [33566, 2.3599803135766013, 15.087175757188202, 3.8842213836479766],
 [12435, 1.413389406683337, 1.1681635289716643, 1.0808161402253689],
 [3118, 1.35801393728223, 0.6819305503284001, 0.8257908635534787],
 [393, 1.1457725947521866, 0.30528096286411277, 0.5525223641302791]]

In [134]:
def get_avg_weights_dict(count_details, average_weights):
    avg_weights_dict = {}
    details = {'count': 0, 'mean': 1, 'var': 2, 'std': 3, 'weight': 4}
    index_transition_dict = {'seen': 0, 'crf': 1, 'mo': 2, 'vb': 3, 'vd': 4, 'sold': 5}
    for key, index in index_transition_dict.items():
        avg_weights_dict[key] = {}
        for detail, value in details.items():
            if value == 4:
                avg_weights_dict[key][detail] = average_weights[index]
            else:
                avg_weights_dict[key][detail] = count_details[index][value]
                
    final_dict = {'benchmark': avg_weights_dict}
    return final_dict

In [135]:
avg_weights_dict = get_avg_weights_dict(required_data, average_weights)

In [136]:
avg_weights_dict

{'benchmark': {'seen': {'count': 425491,
   'mean': 8.662099712953726,
   'var': 184.1205091719551,
   'std': 13.569101266183958,
   'weight': 1.6897012565994471},
  'crf': {'count': 60834,
   'mean': 1.7770053163521644,
   'var': 2.923983717504014,
   'std': 1.7099659989321467,
   'weight': 4.094615360095297},
  'mo': {'count': 33566,
   'mean': 2.3599803135766013,
   'var': 15.087175757188202,
   'std': 3.8842213836479766,
   'weight': 11.602898472790772},
  'vb': {'count': 12435,
   'mean': 1.413389406683337,
   'var': 1.1681635289716643,
   'std': 1.0808161402253689,
   'weight': 16.30778956059768},
  'vd': {'count': 3118,
   'mean': 1.35801393728223,
   'var': 0.6819305503284001,
   'std': 0.8257908635534787,
   'weight': 76.46804805825445},
  'sold': {'count': 393,
   'mean': 1.1457725947521866,
   'var': 0.30528096286411277,
   'std': 0.5525223641302791,
   'weight': 296.4618874480882}}}

In [137]:
average_prob_data

[[0.5279314471649116,
  0.35312105150144185,
  0.10458844461162081,
  0.0027177272684323415,
  0.01141612431042796,
  0.00022520514315489288],
 [0.6507207348073527,
  0.06058925795815044,
  0.045531498253299066,
  0.2316192524026409,
  0.011451533450871715,
  8.772312768340108e-05],
 [0.7268503623441341,
  0.13169685735454117,
  0.10187637665619122,
  0.028272218006492,
  0.011251454147395656,
  5.273149124657245e-05],
 [0.794586472539281,
  0.1262060852211035,
  0.05553300914826999,
  0.010744147668022412,
  0.012591144000981463,
  0.00033914142234214975],
 [0.4847052845528458,
  0.08046602787456447,
  0.06784988385598141,
  0.003048780487804878,
  0.12352227476356395,
  0.2404077484652396],
 [0.815840621963071,
  0.08892128279883382,
  0.028182701652089404,
  0.0029154518950437317,
  0.04956268221574344,
  0.014577259475218658]]

In [138]:
avg_transistion_count

[[38,
  8,
  1,
  16,
  15,
  3,
  3,
  3,
  2,
  4,
  11,
  3,
  16,
  4,
  4,
  5,
  8,
  4,
  4,
  3,
  14,
  7,
  11,
  3,
  9,
  5,
  12,
  37,
  47,
  3,
  5,
  7,
  1,
  29,
  5,
  1,
  5,
  4,
  3,
  2,
  39,
  41,
  96,
  10,
  3,
  4,
  6,
  4,
  3,
  3,
  6,
  3,
  3,
  5,
  4,
  8,
  7,
  3,
  5,
  9,
  10,
  4,
  10,
  29,
  6,
  16,
  24,
  3,
  1,
  3,
  6,
  2,
  11,
  42,
  1,
  2,
  3,
  38,
  3,
  9,
  77,
  4,
  29,
  3,
  6,
  7,
  13,
  6,
  19,
  1,
  17,
  5,
  4,
  10,
  2,
  15,
  25,
  1,
  8,
  2,
  23,
  19,
  7,
  4,
  2,
  6,
  27,
  41,
  54,
  5,
  5,
  1,
  23,
  3,
  3,
  9,
  13,
  10,
  2,
  40,
  8,
  1,
  7,
  11,
  263,
  1,
  20,
  5,
  16,
  13,
  5,
  1,
  2,
  2,
  11,
  16,
  29,
  4,
  20,
  4,
  15,
  8,
  4,
  4,
  16,
  1,
  46,
  23,
  6,
  24,
  5,
  18,
  4,
  49,
  7,
  8,
  13,
  3,
  3,
  7,
  5,
  10,
  55,
  14,
  4,
  15,
  5,
  27,
  3,
  4,
  3,
  6,
  9,
  8,
  1,
  2,
  12,
  5,
  90,
  4,
  23,
  3,
  2,
  3,
  25,
  33,
  

In [139]:
def get_weights(prob_data):
    """
    Equation is of form, AX = B
    lhs: AT * A * X
    rhs: AT * B
    Returns 1/Solution of Equation, X: Weights
    """
    prob_matrix = np.array(prob_data)
    states = prob_matrix.shape[0]
    eigvalmat = (prob_matrix - scipy.sparse.eye(states)).T
    prob_distribution_constraint = scipy.ones((1, states))
    lhs = scipy.sparse.vstack((eigvalmat, prob_distribution_constraint))
    rhs = np.zeros(states + 1)
    rhs[-1] = 1
    stationary_distribution = scipy.sparse.linalg.lsqr(lhs, rhs)
    weights = 1 / stationary_distribution[0]
    weights = weights.tolist()
    
    return weights

In [140]:
average_weights = get_weights(average_prob_data)

In [141]:
average_weights

[1.6897012565994471,
 4.094615360095297,
 11.602898472790772,
 16.30778956059768,
 76.46804805825445,
 296.4618874480882]

In [117]:
total_weights_data = []
for data in trans_prob_data:
    total_weights_data.append(get_weights(data))

In [118]:
total_weights_data

[[1.684210526315811,
  4.745134383688787,
  -104881364060617.64,
  5.114885114885334,
  -104881364060617.64,
  -104881364060617.64],
 [1.1428571428571415,
  7.999999999999911,
  948126237341157.0,
  948126237341157.0,
  948126237341157.0,
  948126237341157.0],
 [289791866771305.1,
  375299968947541.3,
  0.999999999999998,
  375299968947541.3,
  375299968947541.3,
  375299968947541.3],
 [1.3333333333333282,
  3.9999999999999476,
  277812410748637.8,
  277812410748637.8,
  277812410748637.8,
  277812410748637.8],
 [1.000000000000001,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0],
 [1.000000000000001,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0],
 [1.000000000000001,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0],
 [1.000000000000001,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,


In [142]:
visit_transitions_data = visit_order_data['visit_transitions'].tolist()

In [143]:
len(visit_transitions_data)

49121

In [144]:
total_transistions_count = []
for transistions in visit_transitions_data:
    count = [transistions.count(1), transistions.count(2), transistions.count(3), transistions.count(4), transistions.count(5), transistions.count(6)]
    total_transistions_count.append(count)

In [145]:
total_transistions_count

[[38, 13, 0, 12, 0, 0],
 [8, 1, 0, 0, 0, 0],
 [1, 0, 2, 0, 0, 0],
 [16, 5, 0, 0, 0, 0],
 [15, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0],
 [2, 0, 1, 0, 1, 0],
 [4, 0, 0, 0, 0, 0],
 [11, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0],
 [16, 0, 0, 0, 0, 0],
 [4, 1, 0, 0, 0, 0],
 [4, 0, 0, 0, 0, 0],
 [5, 1, 0, 0, 0, 0],
 [8, 2, 0, 1, 1, 1],
 [4, 0, 0, 0, 0, 0],
 [4, 2, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0],
 [14, 0, 4, 0, 0, 0],
 [7, 2, 0, 0, 0, 0],
 [11, 1, 0, 1, 0, 0],
 [3, 1, 0, 0, 0, 0],
 [9, 2, 1, 0, 0, 0],
 [5, 0, 0, 0, 0, 0],
 [12, 0, 0, 0, 0, 0],
 [37, 0, 4, 0, 0, 0],
 [47, 1, 5, 0, 0, 0],
 [3, 1, 0, 0, 0, 0],
 [5, 1, 0, 0, 0, 0],
 [7, 1, 0, 0, 0, 0],
 [1, 2, 3, 1, 0, 0],
 [29, 8, 3, 4, 0, 0],
 [5, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 0],
 [5, 0, 0, 0, 0, 0],
 [4, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0],
 [2, 0, 2, 0, 0, 0],
 [39, 0, 8, 0, 0, 0],
 [41, 0, 0, 0, 0, 0],
 [96, 2, 0, 0, 0, 0],
 [10, 3, 0, 0, 1, 0],
 [3, 2, 0, 0, 0, 0],
 [4, 0, 0, 0, 0, 0],
 [6, 2, 0, 0, 0, 

In [146]:
buyer_ids = visit_order_data['buyer_id'].tolist()

In [147]:
#buyer_ids

In [148]:
final = list(map(list.__add__, total_weights_data, total_transistions_count)) 

In [149]:
final

[[1.684210526315811,
  4.745134383688787,
  -104881364060617.64,
  5.114885114885334,
  -104881364060617.64,
  -104881364060617.64,
  38,
  13,
  0,
  12,
  0,
  0],
 [1.1428571428571415,
  7.999999999999911,
  948126237341157.0,
  948126237341157.0,
  948126237341157.0,
  948126237341157.0,
  8,
  1,
  0,
  0,
  0,
  0],
 [289791866771305.1,
  375299968947541.3,
  0.999999999999998,
  375299968947541.3,
  375299968947541.3,
  375299968947541.3,
  1,
  0,
  2,
  0,
  0,
  0],
 [1.3333333333333282,
  3.9999999999999476,
  277812410748637.8,
  277812410748637.8,
  277812410748637.8,
  277812410748637.8,
  16,
  5,
  0,
  0,
  0,
  0],
 [1.000000000000001,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  15,
  0,
  0,
  0,
  0,
  0],
 [1.000000000000001,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  -948126237341157.0,
  3,
  0,
  0,
  0,
  0,
  0],
 [1.000000000000001,
  -9481262373

In [150]:
columns = ['seen_weight', 'crf_weight', 'mo_weight', 'vb_weight', 'vd_weight', 'sold_weight', 'seen_count', 'crf_count', 'mo_count', 'vb_count', 'vd_count', 'sold_count']

In [151]:
df = pd.DataFrame(final, columns=columns)

In [152]:
df['buyer_id'] = buyer_ids

In [153]:
df.set_index('buyer_id', inplace = True) 

In [154]:
df

,seen_weight,crf_weight,mo_weight,vb_weight,vd_weight,sold_weight,seen_count,crf_count,mo_count,vb_count,vd_count,sold_count
buyer_id,,,,,,,,,,,,
418415,1.684211e+00,4.745134e+00,-1.048814e+14,5.114885e+00,-1.048814e+14,-1.048814e+14,38,13,0,12,0,0
447115,1.142857e+00,8.000000e+00,9.481262e+14,9.481262e+14,9.481262e+14,9.481262e+14,8,1,0,0,0,0
473455,2.897919e+14,3.753000e+14,1.000000e+00,3.753000e+14,3.753000e+14,3.753000e+14,1,0,2,0,0,0
473489,1.333333e+00,4.000000e+00,2.778124e+14,2.778124e+14,2.778124e+14,2.778124e+14,16,5,0,0,0,0
473493,1.000000e+00,-9.481262e+14,-9.481262e+14,-9.481262e+14,-9.481262e+14,-9.481262e+14,15,0,0,0,0,0
473547,1.000000e+00,-9.481262e+14,-9.481262e+14,-9.481262e+14,-9.481262e+14,-9.481262e+14,3,0,0,0,0,0
473633,1.000000e+00,-9.481262e+14,-9.481262e+14,-9.481262e+14,-9.481262e+14,-9.481262e+14,3,0,0,0,0,0
473689,1.000000e+00,-9.481262e+14,-9.481262e+14,-9.481262e+14,-9.481262e+14,-9.481262e+14,3,0,0,0,0,0
473792,2.555556e+00,4.600000e+01,3.285714e+00,4.600000e+01,4.600000e+00,4.600000e+01,2,0,1,0,1,0


In [155]:
df['seen'] = df[['seen_count', 'seen_weight']].values.tolist()
df['crf'] = df[['crf_count', 'crf_weight']].values.tolist()
df['mo'] = df[['mo_count', 'mo_weight']].values.tolist()
df['vb'] = df[['vb_count', 'vb_weight']].values.tolist()
df['vd'] = df[['vd_count', 'vd_weight']].values.tolist()
df['sold'] = df[['sold_count', 'sold_weight']].values.tolist()

In [156]:
d = df.drop(columns, axis = 1) 

In [157]:
k = d.to_dict('index')

In [158]:
k

{418415: {'seen': [38.0, 1.684210526315811],
  'crf': [13.0, 4.745134383688787],
  'mo': [0.0, -104881364060617.64],
  'vb': [12.0, 5.114885114885334],
  'vd': [0.0, -104881364060617.64],
  'sold': [0.0, -104881364060617.64]},
 447115: {'seen': [8.0, 1.1428571428571415],
  'crf': [1.0, 7.999999999999911],
  'mo': [0.0, 948126237341157.0],
  'vb': [0.0, 948126237341157.0],
  'vd': [0.0, 948126237341157.0],
  'sold': [0.0, 948126237341157.0]},
 473455: {'seen': [1.0, 289791866771305.1],
  'crf': [0.0, 375299968947541.3],
  'mo': [2.0, 0.999999999999998],
  'vb': [0.0, 375299968947541.3],
  'vd': [0.0, 375299968947541.3],
  'sold': [0.0, 375299968947541.3]},
 473489: {'seen': [16.0, 1.3333333333333282],
  'crf': [5.0, 3.9999999999999476],
  'mo': [0.0, 277812410748637.8],
  'vb': [0.0, 277812410748637.8],
  'vd': [0.0, 277812410748637.8],
  'sold': [0.0, 277812410748637.8]},
 473493: {'seen': [15.0, 1.000000000000001],
  'crf': [0.0, -948126237341157.0],
  'mo': [0.0, -948126237341157.0],

In [173]:
k['benchmark']

KeyError: 'benchmark'

In [159]:
import copy
p = copy.deepcopy(k)

In [160]:
len(p)

49121

In [161]:
# for key, value in p.items():
#     for transition, data in value.items():
#         if data[0] == 0:
#             del k[key][transition]

In [162]:
def remove_irrelevant_data(total_weights_dict):
    total_weights_dict_copy = copy.deepcopy(total_weights_dict)
    for key, value in total_weights_dict_copy.items():
        for transition, data in value.items():
            if data[0] == 0:
                del total_weights_dict[key][transition]

    return total_weights_dict

In [163]:
# for key, value in k.items():
#     if bool(k[key]) == False:
#         del k[key]

In [164]:
 k = remove_irrelevant_data(k)

In [165]:
avg_weights_dict

{'benchmark': {'seen': {'count': 425491,
   'mean': 8.662099712953726,
   'var': 184.1205091719551,
   'std': 13.569101266183958,
   'weight': 1.6897012565994471},
  'crf': {'count': 60834,
   'mean': 1.7770053163521644,
   'var': 2.923983717504014,
   'std': 1.7099659989321467,
   'weight': 4.094615360095297},
  'mo': {'count': 33566,
   'mean': 2.3599803135766013,
   'var': 15.087175757188202,
   'std': 3.8842213836479766,
   'weight': 11.602898472790772},
  'vb': {'count': 12435,
   'mean': 1.413389406683337,
   'var': 1.1681635289716643,
   'std': 1.0808161402253689,
   'weight': 16.30778956059768},
  'vd': {'count': 3118,
   'mean': 1.35801393728223,
   'var': 0.6819305503284001,
   'std': 0.8257908635534787,
   'weight': 76.46804805825445},
  'sold': {'count': 393,
   'mean': 1.1457725947521866,
   'var': 0.30528096286411277,
   'std': 0.5525223641302791,
   'weight': 296.4618874480882}}}

In [166]:
p = {**k, **avg_weights_dict}

In [174]:
p['benchmark']

{'seen': {'count': 425491,
  'mean': 8.662099712953726,
  'var': 184.1205091719551,
  'std': 13.569101266183958,
  'weight': 1.6897012565994471},
 'crf': {'count': 60834,
  'mean': 1.7770053163521644,
  'var': 2.923983717504014,
  'std': 1.7099659989321467,
  'weight': 4.094615360095297},
 'mo': {'count': 33566,
  'mean': 2.3599803135766013,
  'var': 15.087175757188202,
  'std': 3.8842213836479766,
  'weight': 11.602898472790772},
 'vb': {'count': 12435,
  'mean': 1.413389406683337,
  'var': 1.1681635289716643,
  'std': 1.0808161402253689,
  'weight': 16.30778956059768},
 'vd': {'count': 3118,
  'mean': 1.35801393728223,
  'var': 0.6819305503284001,
  'std': 0.8257908635534787,
  'weight': 76.46804805825445},
 'sold': {'count': 393,
  'mean': 1.1457725947521866,
  'var': 0.30528096286411277,
  'std': 0.5525223641302791,
  'weight': 296.4618874480882}}

In [176]:
p[480414]

{'seen': [2.0, 2.0000000000000013], 'mo': [1.0, 2.0000000000000013]}

In [169]:
list = []
for key, value in avg_weights_dict.items():
    list.append([key, value])

In [170]:
list

[['benchmark',
  {'seen': {'count': 425491,
    'mean': 8.662099712953726,
    'var': 184.1205091719551,
    'std': 13.569101266183958,
    'weight': 1.6897012565994471},
   'crf': {'count': 60834,
    'mean': 1.7770053163521644,
    'var': 2.923983717504014,
    'std': 1.7099659989321467,
    'weight': 4.094615360095297},
   'mo': {'count': 33566,
    'mean': 2.3599803135766013,
    'var': 15.087175757188202,
    'std': 3.8842213836479766,
    'weight': 11.602898472790772},
   'vb': {'count': 12435,
    'mean': 1.413389406683337,
    'var': 1.1681635289716643,
    'std': 1.0808161402253689,
    'weight': 16.30778956059768},
   'vd': {'count': 3118,
    'mean': 1.35801393728223,
    'var': 0.6819305503284001,
    'std': 0.8257908635534787,
    'weight': 76.46804805825445},
   'sold': {'count': 393,
    'mean': 1.1457725947521866,
    'var': 0.30528096286411277,
    'std': 0.5525223641302791,
    'weight': 296.4618874480882}}]]

In [200]:
avg_weights_dict.values

<function dict.values>

In [236]:
s = pd.DataFrame(avg_weights_dict['benchmark'])

In [237]:
s

,seen,crf,mo,vb,vd,sold
count,429189.000000,68098.000000,33963.000000,12586.000000,3288.000000,406.000000
mean,8.543625,1.872059,2.352660,1.405786,1.339308,1.146893
std,13.516439,1.793546,3.870779,1.064794,0.804356,0.548139
var,182.694121,3.216807,14.982932,1.133785,0.646989,0.300456
weight,1.736600,3.818638,11.979236,16.727819,65.127430,272.047821


In [238]:
s

,seen,crf,mo,vb,vd,sold
count,429189.000000,68098.000000,33963.000000,12586.000000,3288.000000,406.000000
mean,8.543625,1.872059,2.352660,1.405786,1.339308,1.146893
std,13.516439,1.793546,3.870779,1.064794,0.804356,0.548139
var,182.694121,3.216807,14.982932,1.133785,0.646989,0.300456
weight,1.736600,3.818638,11.979236,16.727819,65.127430,272.047821


In [239]:
s.index

Index(['count', 'mean', 'std', 'var', 'weight'], dtype='object')

In [240]:
s['index'] = s.index

In [241]:
s

,seen,crf,mo,vb,vd,sold,index
count,429189.000000,68098.000000,33963.000000,12586.000000,3288.000000,406.000000,count
mean,8.543625,1.872059,2.352660,1.405786,1.339308,1.146893,mean
std,13.516439,1.793546,3.870779,1.064794,0.804356,0.548139,std
var,182.694121,3.216807,14.982932,1.133785,0.646989,0.300456,var
weight,1.736600,3.818638,11.979236,16.727819,65.127430,272.047821,weight


In [242]:
s.values.tolist()

[[429189.0, 68098.0, 33963.0, 12586.0, 3288.0, 406.0, 'count'],
 [8.543624962675425,
  1.8720585001099626,
  2.3526600166251037,
  1.4057857701329164,
  1.3393075356415478,
  1.1468926553672316,
  'mean'],
 [13.516438900512771,
  1.7935458435348122,
  3.8707792631639126,
  1.0647935746145833,
  0.8043559675478091,
  0.5481390755130019,
  'std'],
 [182.69412055129487,
  3.216806692861001,
  14.982932104139762,
  1.133785356540502,
  0.6469885225297721,
  0.30045644610424843,
  'var'],
 [1.7366003384268147,
  3.8186376538501694,
  11.979236327854666,
  16.72781882744118,
  65.12743033307481,
  272.0478208544288,
  'weight']]

In [243]:
list_of_lists = s.tolist()

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [244]:
list_of_lists

[[429189.0, 68098.0, 33963.0, 12586.0, 3288.0, 406.0],
 [8.543624962675425,
  1.8720585001099626,
  2.3526600166251037,
  1.4057857701329164,
  1.3393075356415478,
  1.1468926553672316],
 [13.516438900512771,
  1.7935458435348122,
  3.8707792631639126,
  1.0647935746145833,
  0.8043559675478091,
  0.5481390755130019],
 [182.69412055129487,
  3.216806692861001,
  14.982932104139762,
  1.133785356540502,
  0.6469885225297721,
  0.30045644610424843],
 [1.7366003384268147,
  3.8186376538501694,
  11.979236327854666,
  16.72781882744118,
  65.12743033307481,
  272.0478208544288]]

In [245]:
new_row = pd.DataFrame({'index': 'interaction', 'seen':'seen', 'crf':'crf', 
                        'mo':'mo', 'vb':'vb', 'vd':'vd', 
                        'sold': 'sold'}, index =[0]) 
  
s = pd.concat([new_row, s]).reset_index(drop = True) 

/home/pavank/miniconda3/envs/dsenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """


In [246]:
s

,crf,index,mo,seen,sold,vb,vd
0,crf,interaction,mo,seen,sold,vb,vd
1,68098,count,33963,429189,406,12586,3288
2,1.87206,mean,2.35266,8.54362,1.14689,1.40579,1.33931
3,1.79355,std,3.87078,13.5164,0.548139,1.06479,0.804356
4,3.21681,var,14.9829,182.694,0.300456,1.13379,0.646989
5,3.81864,weight,11.9792,1.7366,272.048,16.7278,65.1274
